# Remove Arabic sentences

In [3]:
import pandas as pd
import nltk
import re

In [230]:
# Checking removing Arabic sentences
data = pd.read_csv('splitted_reviews_into_sentences_sID.csv') 
df = pd.DataFrame(data, columns=['r_id', 's_id','app_name','app_id', 'date', 'title', 'review', 'rating'])

In [217]:
# Arabic review > df['review'][14614] 2760
# Review with Emoji > df['review'][727]
# Review with numbers > df['review'][726]
sent = df['review'][36]
print(sent)

..لقد طلبت يوم ١٦ يونيو وتم الدفعه من ابل بي بعد دقائق قليله يتم اشعاري بأن عملية الدفع قد فشلت وعند الدفع مره اخرى اتتني رسالة نصيه للدفع مرتين.وعلى الرغم من الدفع مرتين اشعار فشل عملية الدفع مستمر بالظهور ولكن اعلمني مندوب التوصيل بانه تم الدفعه ولا حاجه للدفع مره اخرى وعند التواصل مع الدعم لهنقرستيشن تم  التواصل بالبدايه بسهوله ومن بعد ذالك لم يتم الرد رغم تعاوني التام معهم  واموالي لم يتم استردادها.


In [231]:
reviews_list = df['review'].tolist()
print(reviews_list)

['A very decent app.', 'Does the job pretty well.', 'Hats off to them for delivering in my area not many apps do.', 'The order so far is always accurate however they do take a lot of time sometimes and the food gets a little bit cold, and they are not accepting cash on delivery.', "I do not use use debit/credit card so now I can't order which is very upsetting i would be very happy if they did COD.", 'Overall a very good app.', 'With few amendments definitely a 5 star', 'I ordered my grocery at 2:15, and the driver whatsapp me at 3:05 the app didnt get my permission to share my contact info with the driver.', 'Anyway first he requested me to list the order through whatsapp and I refused.', 'Then one hour later at 4:07 he texted me pictures of different alternatives of my order, so I told him to skip the one that are not available.', 'The he sent me his bank account number requesting for money!!', 'I told him I will pay through the app he needs to submit the pill there but he told me th

In [232]:
len(df)

14619

In [233]:
arabic_sent = 0
for x in range(len(reviews_list)):
    if re.match('[\u0621-\u064A\u0660-\u0669 ]+', reviews_list[x]):
        df.drop(x, inplace=True)
        arabic_sent = arabic_sent + 1
        print(x, reviews_list[x])
        
print("Number of dropped Arabic sentences:", arabic_sent)

53 استخدمت البرنامج عدة مرات اثناء فترة الحظر وبعد الحظر.
54 تطبيق عقيم ومعقد وخدمة توصيل سيئة جدا جدا وخدمة عملاء معدومة.
55 طلب من كل ٣ طلبات عملتها من التطبيق توصلني ناقصة واضطر اجلس مع خدمة العملاء على الشات ساعات وكل مرة ما تتحل مشكلتي ولا ترجع فلوسي واضطر ارفع شكاوى لهيئة الاتصال وتقنية المعلومات.
56 استغرب كيف ابل ستور يسمح لبرامج النصب هذي.
265 مع انهم اخذوا فلوسي كنت اراسلهم وما يردون وخسرت فلوسي.
266 ابدا لا تستخدموا التطبيق.
267 واقسم اني ما راح استخدمه مره ثانيه.
574 اذا تبين تشتكي ارفع يدينك للسما و ادعي ربك اصرفلك من انك تدور وراهم، و من فتره قريبه يقعدون ساعة و هم يدورونلك سايق .
646 تطبيق سئ جدا وخدمه سيئه طلبت و كان المفترض الطلب يوصل بعد نص ساعه .بعد نص ساعه رساله بان الطلب تم الغاؤه .I ordered and everything was fine the order supposed to be delivered after 30 minutes .Instead of delivery I simply just received a message that order canceled or can't be delivered .Refund will be done 😭🤔
664 خدمة عملاء سيئة جدا.. اذا كان ولا بد من الطلب بهذا التطبيق فلا تدفع بالفيزا بل

1713 تواصلت مع الدعم على امل انهم يقدرون العملاء حيث اني من الناس اللي مستحيل ياكل اكل بارد او باهت، طلبت منهم يلغون الطلب قالوا بنلغيه بس فلوسك ما ترجع، قلت لهم الحكايه كامله ومحادثتي مع المندوب وتأكيده بانه ماخذ اكثر من طلب وبرضو سألت الدعم اذا كانت سياسات التطبيق تسمح بان المندوب ياخذ اكثر من طلب وكان الجواب بالتأكيد!
1714 يعني معقوله ندفع فلوس لخدمات سيئه زي كذا؟ أستخدم التطبيق من 2018 ومشارك فيه كتاجر وكشخص، تم اغلاق الاشتراك مع مطعمي وايضاً تم حذف التطبيق.
1715 ليش مافيه تقييم للمطعم والمندوب
1716 اكثر من ثلاث مرات وعلى اخر يوم ما استلمت الطلب وقفلو كل المطاعم والموضف يقول تبي اجيب لك فطيره من البقاله!
1717 الفلوس ليه كثير في التوصيل
1718 السائق سيء وضع الاكل عند الباب وكاد يمشي مع العلم اني لم اختر هذه الخدمة
1719 غالي كل شي🥺
1720 كاتبين كود خصم ٣٠٪؜ وبعدين تخصموا ١٥ ريال من طلب قيمته ١٧١ ريال!
1722 سبق وذكرت أن كثير من الأسعار مرتفعة عن المطاعم ويفضل مراجعتها، سعر الكباب في تنور اسطنبول 20 ريال وعن طريق تطبيقكم 27 ريال يعني ماخذين 7 ريال زيادة وفوقها قيمة التوصيل وهذا مجرد مثال

1833 وداعااااا ولا انصح بأستخدام تطبيق هنقرستيشن..والحمدلله ان الاختيار في التطبيقات الاخرى متوفر وبشكل افضل بالخدمه و القيمه .
1834 يوصل الاكل بارد بسبب تاخر المندوب
1835 تطبيق سيئ بمعنى الكلمة خدمة عملاء غير محترفة وجودة طعام من مطاعم سيئة ومبالغ ندفعها في خدمات لا تقدم بشكل جيد للاسف
1836 انا كنت استخدم تطبيق هنقرستيشن  ولاكن جاء تحديث لازم احدث او ماقدر ادخل التطبيق  والتحديث لل ios13  وانا لسا ios12.5  وهذا اشوف انه ظلم المفروض التحديث مو اجباري
1837 اول طلب ليا في التطبيق يقول توصيل مجاني بس لما اجي ادفع يطلب حق التوصيل 😵‍💫
1838 موجود رصيد والسواق يقول ما فيه رصيد
1839 والله التطبيق صراحتا في البداية معي كان مرة ممتاز بس كل ما طلبت منه اكثر صرت الاحظ اغلاط اكثر يا الطلب يجي متأخر يا بعض الاصناف من الطلب ما يجيك يا الطلب بكبره يكون غلط ولما تطلب المساعدة منهم يحطونك في قائمة انتظار يتوقعون منك تطفش و تروح والله خذولني صراحتا اخر مرة رح استخدمه يعطيكم العافية.
1840 يكذبون باسعار المنيو تجيك اغلى من المطعم نفسه تطبيق سيء واتمنى يلتفتون له ويلغونه
1841 طلبت مرتين في نفس اليوم و ما وص

1948 للأسف طلبت منهم ب ٦٠ ريال وبعد ٤٠ دقيقه تواصلت مع المندوب وقال اعتذر انشغلت ما اقدر اجيب الطلب!!
1949 ولا قدرت اكنسل الا بالعافية ويوم طلبت إرجاع المبلغ قالو محتاجين ٢١ يوم عمل والى الان ٣ شهور ولا حياه لمن تنادي.. كل فلوسهم رايحه شكلها إعلانات بس..اما مستوى الخدمه👎🏻
1950 التطبيق ما يخليك تتواصل معهم والطلبات تطول ما عرفت ان هذا التطبيق خايس الا يوم شفت معامله( جاهز) خدمه عملاء ولا اروع وتواصل سريع
1951 التتبع غير دقيق ، ماتقدر تتواصل مع السواق او الشركة بسهولة ، سعر توصيل مرتفع ، ويوصل طلبين في وقت واحد ، يعني مستحيل يجي طلبك بسرعة او ساخن .
1952 عندي خدمة توصيل مجاناً ثلاثة مرات لمدة خمس ايام وماشفت شي إنصب عليه
1953 تطبيق سيء والمندوب ما يتواصل معك وخدمة العملاء ما يردون عليك
1954 ٨٠٪؜ من العاملين بتوصيل من الجنسيه الاسيويه , والاكل يوصل بحاله سئه جدا جدا ويصل احينن وهو مفكوك ومتلاعب به من قبل المندوب.
1956 المطعم جيد وخدمة المشرف محمد الحسيني رائعة
1957 طلب المندوب فلوس توصيل زياده غير اللي تم دفعها في التطبيق
1958 انا ☝️
1959 غاليه جدا فيه تطبيقات ارخص
1960 تطبيق جدا جدا سيئ


2056 أسعار التوصيل عندكم مرة مرتفعة
2057 اسوء تطبيق خدمه عملاء ماترد و اذا يردون مايعطون الاجابه الشافيه و الطلب صار ياخذ ساعه وزياده وفوقها يتأخر المندوب بعد
2058 اسعار التوصيل تزود كل فتره مادري وين بيوقف حد التوصيل والمنيو اغلى من المطعم بمنيو هنقريستيشن
2059 سعر التوصيل غالي جداً
2060 اسوء برنامج توصيل ممكن تجربه طلبت ودفعت اون لاين والمندوب عيى يوصلي الطلب وادق على الدعم محد يجاوبني
2061 محد يرد والاسعار كل مالها تصير اغلى واغلى بكل وقاحة ووضوح اسوء تطبيق
2062 اخر تجربه معاهم كانت فاشله جدا طلبت من مطعم ودفعت وراح المندوب والمطعم ماكان عنده الطلب والمندوب محد يرد عليه من شركة هنقرستيشن لحل المشكله ولا انا كا عميل يردون علي وراحت فلوسي هباء منثورا + لما اجي اسوي شكوى يطلع حدث خطا اتمنى ترجع فلوسي
2063 مثلا طلب من شاورمر عربو 6 قطع .سعرها بالتطبيق : 32 ﷼.سعرها بالمطعم : 26 ﷼ .وفوق هذا اخذ توصيل : 19 ﷼..وطلبت من ماكدونالدز وكان السعر تقريبا 230 ﷼ مع التوصيل عن طريق هانقرستيشن ومن فضل ربي علق التطبيق وما قدرنا نطلب منه وحولنا لتطبيق جاهز الطلب كامل مع توصيله ما وصل 190 ﷼..وتأكدوا من ك

...

12244 لايوجد مندوبين لاستقبال الطلبات تأخير
12245 توني طالب منه طول علي مجاني طلب ثاني وبارد وقعد حوالي ساعة ونص عشان يوصل
12246 ما انصح اي شخص يطلب من عندهم الطلب ثلاث ساعات قسم بالله وصور المحادثة موجودة
12247 اسوء برنامج توصيل ياليت يبغونه او يحطو عليه مشرفين صح .ما يستاهل ولا نجمة
12248 خدمة سيئه جداً مايستاهل التحميل
12249 سيئ
12250 بطيئيننن مرااا وخدمة عملاء سيئه ولا يردون بسرعه
12251 اسوء من السيء وع
12252 لحد يحمله والله بيندم لا الدعم الفني متجاوب ولا فيه مناديب كثير بغض النظر عن كون القائمين على التطبيق غير سعوديين وتطلب الطلب ومايجيك الا بعد طلعت روح المهم اتمنى لو جربت التطبيق انك تقيمه هنا عشان يعرفون الناس انه مايستاهل حتى نجمه وحده
12253 التوصيل برسوم ١٧ ريال وليس كما هو موضح في تطبيقكم ان التوصيل مجاني
12254 غالي  ومافيه مطاعم 👎
12255 اكثر من مرة اطلب ويلغي الطلب.
12256 عدم التجاوب مع العمييل .المندوب لو 3ساعات ماوصل الطلب؟!
12257 النجمه كثيره عليييه لو فيه اقل من النجمه حطيت👎🏽
12258 جربته مرتين كان كويس .الين جاني ذاك الواحد الي كرهني حياتي اول شي المطعم عليه عرض توصيل

12358 برنامج سيئ جدا وما فيه تقييم والمناديب اغلبهم سراقين وحسبي الله ونعم الوكيل
12359 مفروض تطبيق توصيل لابد من إضافة وسائل دفع متعددة ابل باي مدى وغيرها من وسائل الدفع كذلك خدمة عملاء تكون متفاعلة مع العميل ثالثا مفروض يكون حزم مع المناديب الكثير يرفض يقبل دفع غير الكأش وهذا سيئ وعدم ثقة بالتطبيق .
12360 بصراحه حلو وقام ينافس شركات كبار توصيل سريع و سهل التعامل معهم ولبقين 👏🏼
12361 ابداً ما اعجبني ماقدرت لا اسدد مدى ولا ابل باي ولما كلمتهم قالوا ابل باي متعطل ومدى فشل عندي واللي ياكلها المندوب
12362 الكل يطلب كاش او تحويل على حساب خاص !!
12363 او يرفض التوصيل !!
12365 تطبيق سيئ والمناديب عندهم حراميه لا أحد يفكر يتعامل معهم
12366 التطبيق سيئ جدا وهناك اربع موظفين لم يخدموني بالشكل الصحيح وتم التواصل مع خدمه العملاء ولم يتعونو معي
12367 برنامج فاشل
12368 البرنامج معقد مو واضح
12369 اتمنى اضافة خدمة تتبع الشقردي الى الآن لااعلم هل الشقردي في طريقه للمطعم ام لا
12370 ضرورري تضيفون الدفع عند طريق الابل باي مررررره ناقص التطبيق بدونه .نضطر نستخدم تطبيقات ثانيه لانها اسهل بالدفع
12371 تطب

12478 كان من الأجدر اختيار صاحب محتوى هادف يمثل السعوديه لأعلان لتطبيق سعودي ومسمى سعودي بحت وسيتم حذف التطبيق
12479 عجزت ادفع ب ابل باي والموقع بطيئ وكل شوي يلغي لحاله بعد ما يقبل ومو رخيص ابدا يحتاج تطوير يحتاج سرعة في التطبيق
12480 سيئ جدً جدً يقول الخدمة غير متوفره
12481 يعلق بايخ يرفع الضغط
12482 فاشل
12483 تطلب من مطعم ماتلقى مندوب يقبل ماادري ليش مسوين التطبيق بصراحه
12484 اسوأ برنامج تعاملت معه وما اتوقع اللي حطه آدمي🐖
12485 تطبيق سيء
12486 تطبيق سيء
12487 برافو عليهم بعد تحديث البرنامج ، يخدم خصوصاً مع الحظر الكلي في مكة وسريع ما شاءالله برافو عليكم كتطبيق عربي👌🏻👌🏻 ،ويقبل الدفع كاش اذا ما اضفت في المحفظه انك تفضل تحويل⭐️⭐️⭐️⭐️
12488 يوم احتجناكم ماخدمتونا وقفلتو التوصيل عن الحي
12489 تطبيق فاشل
12490 لا يوجد خدمه هونيك
12491 ليش قفلتوه..انا معتمد بعدالله على البرنامج ، و وقفوه
12492 حراااميه والمندوب احوله الفلوس ويسكر تليفوناته ومايوصل الطلب واكلم خدمه العملاء يقولي دي مشكلتك انك حولتله وهنرفعلك طلب
12493 يقول مافي تغطية خخخ سلامات لوني عايش فعفيف بس المشكلة اني فالرياض
12494

12600 تطبيقكم فاشلللللل
12601 ميزته بدون عموله لكني للاسف حذفته لانه بطيء جدا وبس يعلق ومافي دعم فني ولا احد يرد على التلفون ولا على المحادثة
12602 لاحول ولاقوة الا بالله
12603 علق لا توجد متاجر
12604 تم الطلب عن طريق التطبيق ثلاثة ايام متتالية وماكان في أي شقردي واحد ولو بالغلط قبل الطلب ... خدمة العملاء سيئة اتواصلت معهم كتابيا وماتم الرد بأي شكل من الأشكال...للأسف اني حملت التطبيق .. وتم الحذف
12605 سيء وبطييييء وجلست عشر دقايق انتظر احد يقبل طلبي؟؟ وهو مايبعد عني الا ٢ كيلو؟ وماحد قبل الطلب وكنسلت!
12606 سيء جداً
12607 اول مره اطلببب و والمندوب بعد ماياخذ الطلب يقول انا مشغول فمشوار وراح اتاخر عليكم والمطعم بيقفل علشان حظر التجول
12608 من اسوء التطبيقات الي مرت علي..اتوقع واضح من طريقة تسويقهم له انه رايح فيها بس قلت بجرب يمكن يصير زي مرسول سنع وللاسف ماله وجه ينقدون على مرسول..ساكنه في شمال الرياض في احد الاحياء المخدومه و اقرب سوبرماركت دقيقتين عني والحي كله مطاعم ومقاهي ومع ذلك يطلع لي مافيه محلات متاحه لك
12609 سيء وجداً وغير صحيح انه يوّصل من والى اي مكان
12610 اسوا برنمج بطي 

14463 ما اتمنى احد ياكلها زي ما اكلتها اليووم.
14464 لكن في مطاعم كثير لما ابحث عنها مب موجودة واضطر ارجع لتطبيق ثاني
14465 الكيكة وصلتني جواتها ثلج؟؟؟؟
14466 تاخير مو طبيعي في الطلبات بفترة رمضان ، مافضل اطلب عن طريقه لانه يحطنا في حرج
14467 لو بس تضيفو فلتر للبحث داخل كل قائمة
14468 نزلت التطبيق وطلبت اول طلبية، وصلت ناقصة.
14469 تواصلت معهم لارجاع المبلغ لحسابي، ولكن تم ارجاع المبلغ الناقص على شكل كوبون في التطبيق بقيمة الطلب الناقص...احتجت طلبية بنص قيمة الكوبون المسترجع وطلبت فيه.
14470 الغريب ان المبلغ المتبقي اختفى من الكوبون، تواصلت معهم للمره الثانية ورد علي الموظف بأن الكوبون يستخدم لمرة واحدة!!
14471 طيب وفلوسي وين تطير؟؟ ..رد علي انه راح يعدل في الكوبون لاسترجاع المبلغ المتبقي واني اقدر استخدمه الان، قفلت منه وللاسف مارجع لي المبلغ، وتواصلت للمرة الثانية، وكذلك مارجع والثالثة وكذلك مارجع، وبعد رابع محاولة تواصل معهم تم استرجاعة.
14472 استخدمت المتبقي في الكوبون وحذفت التطبيق...العيوب، اول شيء ارجاع المبلغ على شكل كوبون، يعني يجبرني على استخدام التطبيق، من يوم تفشلت مع الناس

In [234]:
df = df.reset_index(drop=True)

In [235]:
len(df)

5386

In [143]:
df['review'][36]

'..لقد طلبت يوم ١٦ يونيو وتم الدفعه من ابل بي بعد دقائق قليله يتم اشعاري بأن عملية الدفع قد فشلت وعند الدفع مره اخرى اتتني رسالة نصيه للدفع مرتين.وعلى الرغم من الدفع مرتين اشعار فشل عملية الدفع مستمر بالظهور ولكن اعلمني مندوب التوصيل بانه تم الدفعه ولا حاجه للدفع مره اخرى وعند التواصل مع الدعم لهنقرستيشن تم  التواصل بالبدايه بسهوله ومن بعد ذالك لم يتم الرد رغم تعاوني التام معهم  واموالي لم يتم استردادها.'

In [236]:
table = {"r_id": [], "s_id": [], "app_name": [], "app_id": [], "date": [], "title": [], "review": [], "rating": []}

for x in range(len(df)):
    table["r_id"].append(df['r_id'][x])
    table["s_id"].append(df['s_id'][x])
    table["app_name"].append(df['app_name'][x])
    table["app_id"].append(df['app_id'][x])
    table["date"].append(df['date'][x])
    table["title"].append(df['title'][x])
    table["review"].append(df['review'][x])
    table["rating"].append(df['rating'][x])

In [237]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('reviews_without_Arabic_sentences.csv', index = False)
# This file has been taken to find sentences reference to the extracted features

#  If I want to group the sentences back as reviews

In [224]:
df['review'] = df[['r_id', 's_id','app_name','app_id', 'date', 'title', 'review', 'rating']].groupby(['r_id'])['review'].transform(lambda x: ' '.join(x))

In [225]:
df = df[['r_id','app_name','app_id', 'date', 'title', 'review', 'rating']].drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)

In [226]:
df = df.reset_index(drop=True)

In [229]:
len(df)

2827

In [227]:
table = {"r_id": [], "app_name": [], "app_id": [], "date": [], "title": [], "review": [], "rating": []}

for x in range(len(df)):
    table["r_id"].append(df['r_id'][x])
    table["app_name"].append(df['app_name'][x])
    table["app_id"].append(df['app_id'][x])
    table["date"].append(df['date'][x])
    table["title"].append(df['title'][x])
    table["review"].append(df['review'][x])
    table["rating"].append(df['rating'][x])

In [228]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('reviews_without_Arabic_reviews_sentences.csv', index = False)

# Process sentiment scores

In [238]:
dataS = pd.read_csv('sentences_sentistrength_results.csv') 
df_sentences = pd.DataFrame(dataS, columns=['reference_sentence', 'Positive','Negative'])

In [241]:
sentence_sentiment = []
for x in range(len(df_sentences)):
    if df_sentences['Positive'][x] == abs(df_sentences['Negative'][x]):
        if df_sentences['Positive'][x] == 1:
            sentence_sentiment.insert(x, 0) 
        else:
            sentence_sentiment.insert(x, df_sentences['Negative'][x]) 
    elif df_sentences['Positive'][x] > abs(df_sentences['Negative'][x]):
        sentence_sentiment.insert(x, df_sentences['Positive'][x]) 
    else:
        sentence_sentiment.insert(x, df_sentences['Negative'][x]) 

In [242]:
len(sentence_sentiment)

1165

In [244]:
table = {"sent_id": [], "sentence": [], "sentence_sentiment": []}

for x in range(len(df_sentences)):
    table["sent_id"].append(x)
    table["sentence"].append(df_sentences['reference_sentence'][x])
    table["sentence_sentiment"].append(sentence_sentiment[x])

In [245]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('sentences_sentiment_score.csv', index = False)

In [246]:
df

,sent_id,sentence,sentence_sentiment
0,0,"There is no customer service number, you get l...",0
1,1,Bcs i sent my complaint and there's no respons...,-2
2,2,Surprisingly the customer service only told me...,0
3,3,"Very limited and complicated application, extr...",-4
4,4,.One order out of 3 would reach me missing ite...,-2
...,...,...,...
1160,1160,Till now I have never experienced a late deliv...,0
1161,1161,"Stupid app , i always have problem with discou...",-2
1162,1162,When I first downloaded the chefz app I was ...,2
1163,1163,When I first downloaded the chefz app I was qu...,2


In [247]:
dataR = pd.read_csv('reviews_sentistrength_results.csv') 
df_reviews = pd.DataFrame(dataR, columns=['reference_review', 'Positive','Negative'])

In [249]:
review_sentiment = []
for x in range(len(df_reviews)):
    if df_reviews['Positive'][x] == abs(df_reviews['Negative'][x]):
        if df_reviews['Positive'][x] == 1:
            review_sentiment.insert(x, 0) 
        else:
            review_sentiment.insert(x, df_reviews['Negative'][x]) 
    elif df_reviews['Positive'][x] > abs(df_reviews['Negative'][x]):
        review_sentiment.insert(x, df_reviews['Positive'][x]) 
    else:
        review_sentiment.insert(x, df_reviews['Negative'][x]) 

In [250]:
table = {"review": [], "review_sentiment": []}

for x in range(len(df_reviews)):
    table["review"].append(df_reviews['reference_review'][x])
    table["review_sentiment"].append(review_sentiment[x])

In [251]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('reviews_sentiment_score.csv', index = False)

# Add r_id for SentiStrength files

In [265]:
dataS = pd.read_csv('sentences_sentiment_score.csv') 
df_sentences = pd.DataFrame(dataS, columns=['sentence','sentence_sentiment'])

In [266]:
dataRS = pd.read_csv('reference_sentences.csv') 
df_ref_sentences = pd.DataFrame(dataRS, columns=['r_id', 's_id'])

In [269]:
combined_df = pd.concat([df_ref_sentences, df_sentences], axis=1, join='inner')

In [272]:
combined_df.to_csv('sentences_sentiment_score_with_id.csv', index = False)

In [273]:
dataR = pd.read_csv('reviews_sentiment_score.csv') 
df_reviews = pd.DataFrame(dataR, columns=['review','review_sentiment'])

In [274]:
dataRR = pd.read_csv('reference_reviews.csv') 
df_ref_reviews = pd.DataFrame(dataRR, columns=['r_id'])

In [275]:
combined_dfR = pd.concat([df_ref_reviews, df_reviews], axis=1, join='inner')

In [277]:
combined_dfR.to_csv('reviews_sentiment_score_with_id.csv', index = False)

# Combine sentences and reviews in one file

In [286]:
r_id_from_reviews = combined_dfR['r_id'].tolist()
r_id_from_sentences = combined_df['r_id'].tolist()
sentences_list = combined_df['sentence'].tolist()
reviews_list = combined_dfR['review'].tolist()

In [288]:
combined_dfR

,r_id,review,review_sentiment
0,3,"This app has many issues, technical and person...",-3
1,5,Bcs i sent my complaint and there's no respons...,-4
2,7,.I used this app multiple times during quarant...,-4
3,8,I ordered from store that sell coffee; but in ...,0
4,10,I have 2 delivery addresses added to my accoun...,-2
...,...,...,...
977,10467,I am just overwhelmed with the fast and fur...,-4
978,342,"Stupid app , i always have problem with discou...",-2
979,10132,When I first downloaded the chefz app I was...,2
980,10196,When I first downloaded the chefz app I was ...,2


In [289]:
combined_df

,r_id,s_id,sentence,sentence_sentiment
0,3,31,"There is no customer service number, you get l...",0
1,5,37,Bcs i sent my complaint and there's no respons...,-2
2,5,42,Surprisingly the customer service only told me...,0
3,7,58,"Very limited and complicated application, extr...",-4
4,7,59,.One order out of 3 would reach me missing ite...,-2
...,...,...,...,...
1160,10467,13982,Till now I have never experienced a late deliv...,0
1161,342,899,"Stupid app , i always have problem with discou...",-2
1162,10132,13009,When I first downloaded the chefz app I was ...,2
1163,10196,13212,When I first downloaded the chefz app I was qu...,2


In [290]:
# Combine reviews with sentences
table = {"r_id": [], "review": [], "review_sentiment": [], "s_id": [], "sentence": [], "sentence_sentiment": []}
counter = 0
for x in range(len(r_id_from_sentences)):
    for y in range(len(r_id_from_reviews)):
        if r_id_from_sentences[x] == r_id_from_reviews[y]:
            print("Sentence r_id: ", r_id_from_sentences[x], "equals review r_id: ", r_id_from_reviews[y])
            print("*Sentence", sentences_list[x])
            print("**Review", reviews_list[y])
            print("-------")
            counter = counter + 1
            table["r_id"].append(r_id_from_reviews[y])
            table["review"].append(reviews_list[y])
            table["review_sentiment"].append(combined_dfR['review_sentiment'][y])
            table["s_id"].append(combined_df['s_id'][x])
            table["sentence"].append(sentences_list[x])
            table["sentence_sentiment"].append(combined_df['sentence_sentiment'][x])
        

print("Reference reviews: ", counter)

Sentence r_id:  3 equals review r_id:  3
*Sentence There is no customer service number, you get los whenever you have issue .
**Review This app has many issues, technical and personal as well , first of all never deliver on time, never stick to given promised time. Regarding the personal , they always come to you directly on whats app asking to share location, knowing the app has your full details this is uncomfortable imagine the amount unknown numbers you will hVe . There are times where they don't deliver the order at all, then you have to chase them in twitter otherwise they do not refund your money back , staff attitude is low, they deal with you in not friendly way, weak in arabic and english as well, no respect in dealing  , besides if you live in apartment they refuse to bring it up . Delivery cost is high knowing the app shows only near by restaurant and  still high . There is no customer service number, you get los whenever you have issue . .Honestly one of the worst experien

Sentence r_id:  6121 equals review r_id:  6121
*Sentence As soon as my order got accepted, I saw the driver in another restaurant and I told him not to be late otherwise I won't take the order, then he got out and went to a neighborhood that isn't mine, and of course my order was late because he was busy delivering another order, and I called him and the customer service and refused to take the order and pay for it, but they decided to charge my account on it and now my wallet is on -63 !!
**Review As soon as my order got accepted, I saw the driver in another restaurant and I told him not to be late otherwise I won't take the order, then he got out and went to a neighborhood that isn't mine, and of course my order was late because he was busy delivering another order, and I called him and the customer service and refused to take the order and pay for it, but they decided to charge my account on it and now my wallet is on -63 !! If he's going to break the guidelines by delivering anothe

-------
Sentence r_id:  10076 equals review r_id:  10076
*Sentence If you want fast online food delivery, then choose the Chefz app to speed up your food order process.
**Review    If you want fast online food delivery, then choose the Chefz app to speed up your food order process. The Chefz delivers best quality food at the accurate prices. Highly recommended food delivery app. 
-------
Sentence r_id:  10076 equals review r_id:  10076
*Sentence Highly recommended food delivery app. 
**Review    If you want fast online food delivery, then choose the Chefz app to speed up your food order process. The Chefz delivers best quality food at the accurate prices. Highly recommended food delivery app. 
-------
Sentence r_id:  10078 equals review r_id:  10078
*Sentence Excellent app for food delivery.
**Review   Excellent app for food delivery. I would certainly recommend this online food ordering service because the delivery is super quick and the user experience is too perfect for anyone.  
--

*Sentence  Good food delivery app.
**Review    Good food delivery app. Easy to use and many restaurant choices make experimentation with new things makes it easy.
-------
Sentence r_id:  10426 equals review r_id:  10426
*Sentence Some of my most favorite food delivery app.
**Review    Some of my most favorite food delivery app. Compared with other apps, the Chefz customer service and the delivery service were the best experiences for me.      
-------
Sentence r_id:  10429 equals review r_id:  10429
*Sentence Must have food delivery app.
**Review     Overall the app is best in everything. The food order process becomes very easy with the app. Must have food delivery app.
-------
Sentence r_id:  10430 equals review r_id:  10430
*Sentence The Chefz is the best food delivery app in the UAE.
**Review   Now with the availability to order from the top and elite restaurants in UAE, I think no one will look out for another options. The Chefz is the best food delivery app in the UAE.  
-------


Sentence r_id:  6302 equals review r_id:  6302
*Sentence The best delivery app ever
**Review The best delivery app ever
-------
Sentence r_id:  6345 equals review r_id:  6345
*Sentence the best delivery app in Saudi Arabia
**Review the best delivery app in Saudi Arabia
-------
Sentence r_id:  8107 equals review r_id:  8107
*Sentence The worst delivery app ever they have no honesty and they always late they never come on time the app said orders take between 20 to 35 mins but they take more than an hour even for fast-food restaurants .Also, they do not refund you ever even if the problem was from their side!
**Review The worst delivery app ever they have no honesty and they always late they never come on time the app said orders take between 20 to 35 mins but they take more than an hour even for fast-food restaurants .Also, they do not refund you ever even if the problem was from their side! !.I rather pay more expensive delivery fee from other apps at least I know I will get my food on

...

Sentence r_id:  10256 equals review r_id:  10256
*Sentence    This app provides awesome payment options to the users.
**Review     This app provides awesome payment options to the users. As per your convenience, you can pay instantly on delivery or later.  
-------
Sentence r_id:  10260 equals review r_id:  10260
*Sentence Additionally, the payment options are incredible.
**Review     I 'm pleased with the Chefz App and the deals, and even the order quality is almost correct. Additionally, the payment options are incredible.  
-------
Sentence r_id:  10263 equals review r_id:  10263
*Sentence You can easily order food from the favorite restaurant with flexible payment options.
**Review   Whenever I think of ordering food, I choose Chefz app. You can easily order food from the favorite restaurant with flexible payment options. Best food delivery option in ABU DHABI.
-------
Sentence r_id:  10304 equals review r_id:  10304
*Sentence Additionally, the payment options are incredible.
**Rev

Sentence r_id:  9183 equals review r_id:  9183
*Sentence The worst app ever they don't have even a customer care...أسوء تطبيق توصيل بلا منازع!
**Review The worst app ever they don't have even a customer care...أسوء تطبيق توصيل بلا منازع! .إذا انلغى طلبك من قبل المطعم أو البائع انسى ترجع لك فلوسه؟!!
-------
Sentence r_id:  9365 equals review r_id:  9365
*Sentence Add better options .And the application keeps on hanging and you can't really see where the driver is at..This app needs a lot of improvements and better customer care.
**Review Payment is so inconvenient. Add better options .And the application keeps on hanging and you can't really see where the driver is at..This app needs a lot of improvements and better customer care.
-------
Sentence r_id:  10817 equals review r_id:  10817
*Sentence On two occasions when I had an accidental problem with my order, the Chefz customer care were extremely efficient and kind.
**Review On two occasions when I had an accidental problem with my or

Sentence r_id:  8127 equals review r_id:  8127
*Sentence This is my first time to order with this app and obviously it's the worst app ever, why?
**Review This is my first time to order with this app and obviously it's the worst app ever, why? Because I order from McDonald's with this app and they dedicated 38 SR from me but guess what !! I got nothing my order was at 7:36 am and now it's 8:58 am and I saw the order info it's saying that they are waiting for my payment!?! Really?!?! F this I just want my money back.
-------
Sentence r_id:  8211 equals review r_id:  8211
*Sentence Every thing was bad bad bad bad I will never order on this app ever again this is mu first time and the last there is no employee who respect the customer I really really disappointed with this app ..
**Review Every thing was bad bad bad bad I will never order on this app ever again this is mu first time and the last there is no employee who respect the customer I really really disappointed with this app ..
--

Sentence r_id:  146 equals review r_id:  146
*Sentence I've tried the new order that call it fazhaa .And I've ordered 4 times and none of them I got it also , the last time they took my money and they didn't give me anything, once I took to the delivery he told me hanguer station stoped the order how ever I haven't received anything, where is the justice?
**Review I've tried the new order that call it fazhaa .And I've ordered 4 times and none of them I got it also , the last time they took my money and they didn't give me anything, once I took to the delivery he told me hanguer station stoped the order how ever I haven't received anything, where is the justice?
-------
Sentence r_id:  206 equals review r_id:  206
*Sentence This will be my last time using hunger station.
**Review This will be my last time using hunger station. In case of any issue there is no way to contact anyone for help or support or to even ask a simple question. There are MUCH better apps out there. I won't be wast

Sentence r_id:  6135 equals review r_id:  6135
*Sentence I made an order at 17:25 and waited until 20:00 to find out that the order was never completed and I called the support and had ti wait 27 minutes to answer my call.Support person was polite and promised to return money plus credit of a certain amount that was never added to my account.Additionally the app has glitches which doesn't allow to sign in .I will give them a try today and will return with a review
**Review I made an order at 17:25 and waited until 20:00 to find out that the order was never completed and I called the support and had ti wait 27 minutes to answer my call.Support person was polite and promised to return money plus credit of a certain amount that was never added to my account.Additionally the app has glitches which doesn't allow to sign in .I will give them a try today and will return with a review
-------
Sentence r_id:  10079 equals review r_id:  10079
*Sentence The food delivered by the Chefz app is of t

In [294]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('reviews_sentences_sentiment_scores.csv', index = False)

In [295]:
len(df)

1165

In [293]:
df

,r_id,review,review_sentiment,s_id,sentence,sentence_sentiment
0,3,"This app has many issues, technical and person...",-3,31,"There is no customer service number, you get l...",0
1,5,Bcs i sent my complaint and there's no respons...,-4,37,Bcs i sent my complaint and there's no respons...,-2
2,5,Bcs i sent my complaint and there's no respons...,-4,42,Surprisingly the customer service only told me...,0
3,7,.I used this app multiple times during quarant...,-4,58,"Very limited and complicated application, extr...",-4
4,7,.I used this app multiple times during quarant...,-4,59,.One order out of 3 would reach me missing ite...,-2
...,...,...,...,...,...,...
1160,10467,I am just overwhelmed with the fast and fur...,-4,13982,Till now I have never experienced a late deliv...,0
1161,342,"Stupid app , i always have problem with discou...",-2,899,"Stupid app , i always have problem with discou...",-2
1162,10132,When I first downloaded the chefz app I was...,2,13009,When I first downloaded the chefz app I was ...,2
1163,10196,When I first downloaded the chefz app I was ...,2,13212,When I first downloaded the chefz app I was qu...,2


# Assign sentiment for features

In [20]:
dataF = pd.read_csv('features_with_weight.csv') 
dfF = pd.DataFrame(dataF, columns=['extracted_features', 'meaningful', 'weight'])

In [21]:
dataS = pd.read_csv('reviews_sentences_sentiment_scores.csv') 
dfS = pd.DataFrame(dataS, columns=['r_id', 'review', 'review_sentiment', 's_id', 'sentence', 'sentence_sentiment'])

In [22]:
sentences_list = dfS['sentence'].str.lower().tolist()

In [23]:
for x in range(len(sentences_list)):
    if re.search('costumer service', sentences_list[x]):
        print("Old:", sentences_list[x])
        string = sentences_list[x]
        sentences_list[x] = string.replace('costumer service', 'customer service')
        print("New:", sentences_list[x])

Old: the slowest delivery app with honor degree   beside the worst costumer service
New: the slowest delivery app with honor degree   beside the worst customer service
Old: waste of money and trash costumer service.
New: waste of money and trash customer service.
Old: he said i can just call costumer service for the refund of my money, even refusing to give my order without giving him the cash.
New: he said i can just call customer service for the refund of my money, even refusing to give my order without giving him the cash.
Old: ..i called the costumer service to help me settle this matter and they assured me a refund.
New: ..i called the customer service to help me settle this matter and they assured me a refund.
Old: useless costumer services and you can't find help if you faced an issue with your order..also, recently they have hidden restaurants locations although it's my right as a costumer to choose what's best for me..account and and app have been removed for better options an

In [24]:
# Add the meaningful features to the features list
features_list = []
weight = []
for x in range(len(dfF)):
    if dfF['meaningful'][x] == 1:
        features_list.insert(x, dfF['extracted_features'][x])
        weight.insert(x, dfF['weight'][x])

In [25]:
# Find each features with its reference review/sentence to assign sentiment score
table = {"r_id": [], "review": [], "review_sentiment": [], "s_id": [], "sentence": [], "sentence_sentiment": [], "feature": [], "weight": []}
counter = 0

for x in range(len(features_list)):
    print("-------feature", features_list[x])
    for y in range(len(sentences_list)):
        if features_list[x] in sentences_list[y]:
            counter = counter + 1
            print("Feature:", features_list[x], "weight:", weight[x])
            print("Found in sentence ID #:", dfS['s_id'][y], "|| Sentence::", sentences_list[y])
            table["r_id"].append(dfS['r_id'][y])
            table["review"].append(dfS['review'][y])
            table["review_sentiment"].append(dfS['review_sentiment'][y])
            table["s_id"].append(dfS['s_id'][y])
            table["sentence"].append(sentences_list[y])
            table["sentence_sentiment"].append(dfS['sentence_sentiment'][y])
            table["feature"].append(features_list[x])
            table["weight"].append(weight[x])
        

print(counter)

-------feature customer service
Feature: customer service weight: 52.18833336947196
Found in sentence ID #: 31 || Sentence:: there is no customer service number, you get los whenever you have issue .
Feature: customer service weight: 52.18833336947196
Found in sentence ID #: 37 || Sentence:: bcs i sent my complaint and there's no response from the customer service.
Feature: customer service weight: 52.18833336947196
Found in sentence ID #: 42 || Sentence:: surprisingly the customer service only told me that they will call me later to return the money.
Feature: customer service weight: 52.18833336947196
Found in sentence ID #: 58 || Sentence:: very limited and complicated application, extremely poor delivery service and non-existing customer service.
Feature: customer service weight: 52.18833336947196
Found in sentence ID #: 59 || Sentence:: .one order out of 3 would reach me missing items and i'd be forced to sit through the chat with their customer service but not once was my problem 

Feature: order food weight: 28.17550947222528
Found in sentence ID #: 12897 || Sentence:: i am using this app quite from last 2 months, but every time i order food, i feel that the services are improving.
Feature: order food weight: 28.17550947222528
Found in sentence ID #: 12940 || Sentence:: one of my friends suggested me to do this, and now i can order food of my choice any time anywhere in the uae.
Feature: order food weight: 28.17550947222528
Found in sentence ID #: 13036 || Sentence:: you can review your order, to rate the restaurant and app through which you order food.
Feature: order food weight: 28.17550947222528
Found in sentence ID #: 13101 || Sentence::  if you are looking for an app that offers amazing deals, then you can order food from this app.
Feature: order food weight: 28.17550947222528
Found in sentence ID #: 13104 || Sentence::  when you order food from this app.
Feature: order food weight: 28.17550947222528
Found in sentence ID #: 13105 || Sentence:: you will get 

In [26]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('final_file_with_sentiments.csv', index = False)

# Add feature frequency

In [333]:
data = pd.read_csv('final_file_with_sentiments.csv') 
df = pd.DataFrame(data, columns=['r_id', 'review', 'review_sentiment', 's_id', 'sentence', 'sentence_sentiment', 'feature'])

# frequency regardless sentiment


In [339]:
df['freq'] = df.groupby('feature')['feature'].transform('count')

In [340]:
df

,r_id,review,review_sentiment,s_id,sentence,sentence_sentiment,feature,freq
0,3,"This app has many issues, technical and person...",-3,31,"there is no customer service number, you get l...",0,customer service,192
1,5,Bcs i sent my complaint and there's no respons...,-4,37,bcs i sent my complaint and there's no respons...,-2,customer service,192
2,5,Bcs i sent my complaint and there's no respons...,-4,42,surprisingly the customer service only told me...,0,customer service,192
3,7,.I used this app multiple times during quarant...,-4,58,"very limited and complicated application, extr...",-4,customer service,192
4,7,.I used this app multiple times during quarant...,-4,59,.one order out of 3 would reach me missing ite...,-2,customer service,192
...,...,...,...,...,...,...,...,...
706,10236,What makes theapp best among all its compe...,2,13327,what makes theapp best among all its competito...,2,flexible payment,6
707,10285,"The app is easy to use, and the best part ...",2,13465,"the app is easy to use, and the best part i...",2,flexible payment,6
708,10164,"Whenever I think of ordering food, I choose...",2,13110,you can easily order food from the favorite re...,2,flexible payment,6
709,10185,The chefz app offers various payment optio...,0,13178,"every time i plan to order food online, this a...",0,flexible payment,6


In [341]:
# frequency regardless sentiment
df.to_csv('freq_regardless_sentiment.csv', index = False)

# frequency regarding feature and sentiment

In [336]:
frequency_series = df.groupby(['review_sentiment', 'sentence_sentiment', 'feature']).size()

In [337]:
new_df = frequency_series.to_frame(name = 'freq').reset_index()

In [338]:
new_df

,review_sentiment,sentence_sentiment,feature,freq
0,-5,-5,apple pay,1
1,-5,-5,customer service,3
2,-5,-5,google map,1
3,-5,-2,delivery fee,1
4,-5,-2,free delivery,1
...,...,...,...,...
218,4,4,order food,2
219,4,4,restaurant finder,3
220,4,4,restaurant menu,2
221,4,4,user friendly,3


In [342]:
new_df.to_csv('freq_regard_sentiment.csv', index = False)

In [308]:
# grouped_df = df.groupby(["review_sentiment", "sentence_sentiment", "feature"]).size().reset_index(name="Frequency")

In [309]:
# grouped_df.to_csv('grouped_file.csv', index = False)

In [ ]:
# This is old file without including reviews sentiment score
df.to_csv('features_sentiment_score_with_sent.csv', index = False)